# Using a Temporal Convolutional Network for Daytrading
## Daniel Kalam, Sharvita Paithankar

In [57]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv1D, Dropout, Dense
import pandas as pd
from pandas_datareader import DataReader
import numpy as np
import matplotlib.pyplot as plt
import datetime

## Gathering Data
Getting data for 100 stocks in the date range of April 2nd, 2018 to October 9th, 2020 from yahoo finance.

In [58]:
symbols = ['AAPL', 'TSLA', 'FB', 'GE', 'BRK', 'GOOGL', 'INTC', 'AMD', 'HPE', 'ZM',
          'CAKE', 'AET', 'F', 'KO', 'DDS', 'NVDA', 'NFLX', 'JPM', 'AMZN', 'MSFT']
#TODO: Add 80 more symbols.
source = 'yahoo'
start_date = pd.to_datetime('2018-02-04')
end_date = pd.to_datetime('2020-10-09')
stock_data = {}
for symbol in symbols:
    stock_data[symbol] = DataReader(symbol, source, start_date, end_date)

Create a data frame for each column in a stock's data frame.

In [59]:
open_data = {}
close_data = {}
high_data = {}
low_data = {}
volume_data = {}
adj_close_data = {}

for symbol in stock_data:
    open_data[symbol] = stock_data[symbol].Open
    close_data[symbol] = stock_data[symbol].Close
    high_data[symbol] = stock_data[symbol].High
    low_data[symbol] = stock_data[symbol].Low
    volume_data[symbol] = stock_data[symbol].Volume
    adj_close_data[symbol] = stock_data[symbol]['Adj Close']

In [60]:
open_data['AAPL']

Date
2018-02-05     39.775002
2018-02-06     38.707500
2018-02-07     40.772499
2018-02-08     40.072498
2018-02-09     39.267502
                 ...    
2020-10-05    113.910004
2020-10-06    115.699997
2020-10-07    114.620003
2020-10-08    116.250000
2020-10-09    115.279999
Name: Open, Length: 676, dtype: float64

In [61]:
close_data['AAPL']

Date
2018-02-05     39.122501
2018-02-06     40.757500
2018-02-07     39.884998
2018-02-08     38.787498
2018-02-09     39.102501
                 ...    
2020-10-05    116.500000
2020-10-06    113.160004
2020-10-07    115.080002
2020-10-08    114.970001
2020-10-09    116.970001
Name: Close, Length: 676, dtype: float64

In [62]:
indicator = {}
indicator['AAPL'] = {}
for day in range(0, len(open_data['AAPL'])):
    if open_data['AAPL'][day] < close_data['AAPL'][day]:
        indicator['AAPL'][day] = True
    else:
        indicator['AAPL'][day] = False

indicator['AAPL']

{0: False,
 1: True,
 2: False,
 3: False,
 4: False,
 5: True,
 6: True,
 7: True,
 8: True,
 9: True,
 10: False,
 11: False,
 12: True,
 13: True,
 14: True,
 15: False,
 16: False,
 17: False,
 18: True,
 19: True,
 20: False,
 21: True,
 22: True,
 23: True,
 24: True,
 25: False,
 26: False,
 27: True,
 28: False,
 29: False,
 30: False,
 31: False,
 32: False,
 33: False,
 34: True,
 35: False,
 36: False,
 37: False,
 38: True,
 39: True,
 40: True,
 41: True,
 42: False,
 43: True,
 44: True,
 45: True,
 46: True,
 47: False,
 48: True,
 49: True,
 50: True,
 51: False,
 52: False,
 53: False,
 54: False,
 55: True,
 56: True,
 57: False,
 58: True,
 59: True,
 60: True,
 61: True,
 62: True,
 63: False,
 64: True,
 65: True,
 66: True,
 67: False,
 68: False,
 69: False,
 70: True,
 71: False,
 72: False,
 73: False,
 74: False,
 75: True,
 76: False,
 77: True,
 78: True,
 79: False,
 80: False,
 81: True,
 82: True,
 83: True,
 84: True,
 85: False,
 86: True,
 87: False,
 

In [63]:
indicator = {}
indicator['AAPL'] = {}
for day in open_data['AAPL'].keys():
#     print(day)
    if open_data['AAPL'][day] < close_data['AAPL'][day]:
        indicator['AAPL'][str(day)[0:10]] = True
    else:
        indicator['AAPL'][str(day)[0:10]] = False

indicator['AAPL']

{'2018-02-05': False,
 '2018-02-06': True,
 '2018-02-07': False,
 '2018-02-08': False,
 '2018-02-09': False,
 '2018-02-12': True,
 '2018-02-13': True,
 '2018-02-14': True,
 '2018-02-15': True,
 '2018-02-16': True,
 '2018-02-20': False,
 '2018-02-21': False,
 '2018-02-22': True,
 '2018-02-23': True,
 '2018-02-26': True,
 '2018-02-27': False,
 '2018-02-28': False,
 '2018-03-01': False,
 '2018-03-02': True,
 '2018-03-05': True,
 '2018-03-06': False,
 '2018-03-07': True,
 '2018-03-08': True,
 '2018-03-09': True,
 '2018-03-12': True,
 '2018-03-13': False,
 '2018-03-14': False,
 '2018-03-15': True,
 '2018-03-16': False,
 '2018-03-19': False,
 '2018-03-20': False,
 '2018-03-21': False,
 '2018-03-22': False,
 '2018-03-23': False,
 '2018-03-26': True,
 '2018-03-27': False,
 '2018-03-28': False,
 '2018-03-29': False,
 '2018-04-02': True,
 '2018-04-03': True,
 '2018-04-04': True,
 '2018-04-05': True,
 '2018-04-06': False,
 '2018-04-09': True,
 '2018-04-10': True,
 '2018-04-11': True,
 '2018-04-12

## TensorFlow
### Converting the Data Into Tensors
Turn the data frames into tensorflow datatypes so that they can be processed by tensorflow.

In [65]:
open_train = tf.convert_to_tensor(np.array((10, 637, 1)))
for i in range(0, 10):
    key = list(open_data.keys())[i]
    print(key)
    open_train[i] = tf.convert_to_tensor(np.array(open_data[key]).reshape(len(open_data[key]), 1))
open_aapl = open_data['AAPL']
open_aapl = open_aapl[0 : 637]
open_aapl = np.array(open_aapl).reshape(1, 637, 1)
open_aapl_train = tf.convert_to_tensor(open_aapl)
close_aapl = close_data['AAPL']
close_aapl = close_aapl[0 : 637]
close_aapl = np.array(close_aapl).reshape(1, 637, 1)

AAPL


TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

### Dilated 1D Causal Convolutional Network Class

In [146]:
class Dilated1DCausalConvolutionalNetwork(Conv1D):
    def __init__(self,
                 filters,
                 kernel_size,
                 strides=1,
                 dilation_rate=1,
                 activation=None,
                 use_bias=True,
                 kernel_initializer=None,
                 bias_initializer=tf.zeros_initializer(),
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 trainable=True,
                 name=None,
                 **kwargs):
        super(Dilated1DCausalConvolutionalNetwork, self).__init__(
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding='valid',
            data_format='channels_last',
            dilation_rate=dilation_rate,
            activation=activation,
            use_bias=use_bias,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
            activity_regularizer=activity_regularizer,
            kernel_constraint=kernel_constraint,
            bias_constraint=bias_constraint,
            trainable=trainable,
            name=name,
            **kwargs
        )
    def call(self, inputs):
        padding = (self.kernel_size[0] - 1)*self.dilation_rate[0]
        inputs = tf.pad(inputs, tf.constant([(0, 0), (1, 0), (0, 0)])*padding)
        return super(Dilated1DCausalConvolutionalNetwork, self).call(inputs)

### Temporal Residual Block Class

In [147]:
class TemporalResidualBlock(Layer):
    def __init__(self, n_outputs, kernel_size, strides, dilation_rate, dropout = 0.2,
                trainable=True, name=None, dtype=None, activity_regularizer=None, **kwargs):
        super(TemporalResidualBlock, self).__init__(
            trainable=trainable, name=name, dtype=dtype,
            activity_regularizer=activity_regularizer, **kwargs
        )
        self.dropout = dropout
        self.n_outputs = n_outputs
        self.conv1 = Dilated1DCausalConvolutionalNetwork(n_outputs, kernel_size, strides=strides,
                                                        dilation_rate=dilation_rate, activation=tf.nn.relu,
                                                        name='conv1', trainable=trainable)
        self.conv2 = Dilated1DCausalConvolutionalNetwork(n_outputs, kernel_size, strides=strides,
                                                        dilation_rate=dilation_rate, activation=tf.nn.relu,
                                                        name='conv2', trainable=trainable)
        self.down_sample=None
        
    def build(self, input_shape):
        channel_dim=2
        self.dropout1 = Dropout(self.dropout, [tf.constant(1), tf.constant(1), tf.constant(self.n_outputs)])
        self.dropout2 = Dropout(self.dropout, [tf.constant(1), tf.constant(1), tf.constant(self.n_outputs)])
        if input_shape[channel_dim] != self.n_outputs:
            self.down_sample = Dense(self.n_outputs, activation=None)
    
    def call(self, inputs, training=True):
        x = self.conv1(inputs)
        x = tf.keras.layers.layer_norm(x)
        x = self.dropout1(x, training=training)
        x = self.conv2(x)
        x = tf.keras.layers.layer_norm(x)
        x = self.dropout2(x, training=training)
        if self.down_sample is not None:
            inputs = self.down_sample(inputs)
        return tf.nn.relu(x + inputs)

### Temporal Convolutional Network Class

In [148]:
class TemporalConvolutionalNetwork(Layer):
    def __init__(self, num_channels, kernel_size=2, dropout = 0.2,
                trainable=True, dtype=None, name=None,
                activity_regularizer=None, **kwargs):
        super(TemporalConvolutionalNetwork, self).__init__(
            trainable=trainable, dtype=dtype, name=name,
            activity_regularizer=activity_regularizer, **kwargs)
        self.layers = []
        for i in range(len(num_channels)):
            dilation_size = 2**i
            self.layers.append(TemporalResidualBlock(num_channels[i], kernel_size, strides=1, dilation_rate=dilation_size,
                                     dropout=dropout, name='tblock_{}'.format(i), trainable=trainable))
    def call(self, inputs, training=True):
        outputs = inputs
        for layer in self.layers:
            outputs = layer(outputs, training=training)
        return outputs

### Training the TCN

In [149]:
filter_count = 20 # Amount of filters
filters = [] # Filter size for each residual block
kernel_size = 10 #Resolution of each filter
level = kernel_size
n = 0
while level <= len(open_data0):
    filters.append(filter_count)
    level+=kernel_size + (kernel_size-1)*2**n
    n+=1

In [192]:
day_trading_tcn = TemporalConvolutionalNetwork(filters, kernel_size, dtype='float32')
output = day_trading_tcn(open_aapl_train)

In [194]:
close_aapl = tf.convert_to_tensor(close_aapl)
close_aapl_bool = [int(close_aapl[0, i, 0] > open_aapl_train[0, i, 0]) for i in range(len(close_aapl[0]))]
pred_close_aapl_bool = [int(float(output[0, i, 0]) > open_aapl_train[0, i, 0]) for i in range(len(open_aapl_train[0]))]
np.mean(close_aapl_bool == pred_close_aapl_bool)

0.0

AttributeError: 'TemporalConvolutionalNetwork' object has no attribute 'fit'